#### Открытые данные 

1. Создать приложение, которое будет из готового файла с данными «Сбербанка» (https://www.sberbank.com/ru/analytics/opendata) выводить результат по параметрам:   
    • Тип данных  
    • Интервал дат  
    • Область  
2. Визуализировать выводимые данные с помощью графика

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import widgets, Layout, interact_manual

In [2]:
df = pd.read_csv('opendata.csv', encoding='Windows-1251')
df['date'] = df['date'].astype('datetime64[ns]')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74842 entries, 0 to 74841
Data columns (total 4 columns):
name      74842 non-null object
region    74842 non-null object
date      74842 non-null datetime64[ns]
value     74842 non-null int64
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 2.3+ MB


In [3]:
def show_interactive(regions_value, indicator_value, date_interval):
    start_date = date_interval[0].to_pydatetime()
    end_date = date_interval[1].to_pydatetime()
    #фильтруем согласно условиям
    result_df = df.loc[(df.name == indicator_value) &
                       df.region.isin(regions_value) &
                       (df.date >=  start_date) &
                       (df.date <=  end_date)
                      ]
    #формируему график
    fig, ax = plt.subplots(figsize=(10,10))
    for i, group in result_df.groupby('region'):
        group.plot(x='date', y='value', ax=ax, label=group['region'].iloc[0])
    ax.set_title(indicator_value)
    ax.set_xlabel('Годы')
    plt.show()

In [4]:
start_date = df.date.min()
end_date = df.date.max()

dates = pd.date_range(start_date, end_date, freq='D')

options = [(date.strftime(' %d %b %Y '), date) for date in dates]
index = (0, len(options)-1)

#Создаем интерактивную меню
elem  = interact_manual(show_interactive, 
          regions_value =  widgets.SelectMultiple(
                           description="Выберите регионы",
                           options=df.region.unique(),
                           layout=Layout(width='50%', height='180px'),
                           style = {'description_width': 'initial'}
                           ),
          indicator_value = widgets.Select(
                            description='Укажите показатель',
                            options=df.name.unique(),
                            layout=Layout(width='50%', height='180px'),
                            style = {'description_width': 'initial'}
                            ),
          date_interval =  widgets.SelectionRangeSlider(
                        options=options,
                        index=index,
                        description='Интервал',
                        orientation='horizontal',
                        layout={'width': '600px'},
                        style = {'description_width': 'initial'}
                        )
        )

interactive(children=(SelectMultiple(description='Выберите регионы', layout=Layout(height='180px', width='50%'…